In [95]:
import numpy as np
import sympy as smp
import IPython.display as ipy
from derivativeClass import DerivativeNum4
smp.init_printing(use_latex='mathjax')

STEP = 10e-3
X_LEFT = 0
X_RIGHT = 2

In [96]:
class MaxNorm:
    """Вычисление нормы по правилам пространства С0[a, b]"""

    def __init__(self, xLeft, xRight, func):
        self.xLeft = xLeft
        self.xRight = xRight
        self.f = func
        self.numBlocks = int((xRight - xLeft) / STEP)

    def getX(self):
        return np.linspace(self.xLeft, self.xRight, self.numBlocks)
    
    def calcDistance(self, func1Sym, func2Sym):
        self.f = smp.lambdify(xSym, func1Sym - func2Sym)
        return self.__call__()

    def __call__(self):
        x = self.getX()
        values = []
        
        for i in x:
            values.append(abs(self.f(i)))

        return np.amax(values)


class MaxNorm1(MaxNorm):
    """Вычисление нормы по правилам пространства С1[a, b]"""

    def __call__(self):
        res = MaxNorm.__call__(self)
        self.f = DerivativeNum4(self.f, STEP)
        res += MaxNorm.__call__(self)

        return res


class MaxNorm2(MaxNorm1):
    """Вычисление нормы по правилам пространства С2[a, b]"""

    def __call__(self):
        res = MaxNorm1.__call__(self)
        self.f = DerivativeNum4(DerivativeNum4(self.f, STEP), STEP)
        res += MaxNorm.__call__(self)

        return res

In [97]:
# Тестовые функции

xSym = smp.Symbol('x')
functionsSym = [5 / (2 + 3 * xSym**2),
                2 / (5 + smp.cos(xSym)),
               (3 + 4 * xSym**2)**(1 / 3),
                2 * (smp.pi)**(-1 / 2) * smp.exp(-xSym**2)]

functions = [smp.lambdify(xSym, f) for f in functionsSym]
ipy.display(functionsSym)

⎡                                0.333333333333333             2⎤
⎢   5          2       ⎛   2    ⎞                      -0.5  -x ⎥
⎢────────, ──────────, ⎝4⋅x  + 3⎠                 , 2⋅π    ⋅ℯ   ⎥
⎢   2      cos(x) + 5                                           ⎥
⎣3⋅x  + 2                                                       ⎦

In [98]:
# Пример вычисления нормы пространства С0 для каждой функции на заданном отрезке

funcMaxNorms = [MaxNorm(X_LEFT, X_RIGHT, f) for f in functions]
funcMaxNormsValues = [f() for f in funcMaxNorms]
print(funcMaxNormsValues)

[2.5, 0.43631415071190222, 2.6684016487219422, 1.1283791670955126]


In [99]:
# Пример вычисления нормы пространства С1 для каждой функции на заданном отрезке

funcMaxNorms1 = [MaxNorm1(X_LEFT, X_RIGHT, f) for f in functions]
funcMaxNormsValues1 = [f() for f in funcMaxNorms1]
print(funcMaxNormsValues1)

[4.4885088550574466, 0.52298828976640377, 3.4315410950464722, 2.0961722374250042]


In [100]:
# Пример вычисления нормы пространства С2 для каждой функции на заданном отрезке

funcMaxNorms2 = [MaxNorm2(X_LEFT, X_RIGHT, f) for f in functions]
funcMaxNormsValues2 = [f() for f in funcMaxNorms2]
print(funcMaxNormsValues2)

[25.896019650712063, 0.63624416043042342, 5.025139226652672, 6.4990367853771387]


In [102]:
# Попарные расстояния между функциями в нормированных пространствах С0, С1, С2

norms = [MaxNorm, MaxNorm1, MaxNorm2]
for norm in norms:
    for i in range(len(functionsSym)):
        for j in range(i + 1, len(functionsSym)):
            if(i != j):
                elem = norm(X_LEFT, X_RIGHT, functionsSym[i])
                dist = elem.calcDistance(functionsSym[i], functionsSym[j])
                print(dist)

2.16666666667
2.31125879158
1.3716208329
2.23208749801
0.795045833762
2.64773466337
4.18140129959
4.82816344439
2.55475484565
2.92169230308
1.80240171238
4.27933325276
25.5891628813
27.659436544
20.8491738921
4.51437050934
6.20730868637
10.2010605158
